### Camembert：

Ici, « Candidat », au sens initial, peut désigner chaque fragment d’entité identifié par le modèle,tandis qu’« entité » fait plutôt référence à l’entité nommée complète, après agrégation.

In [ ]:
from transformers import pipeline, TokenClassificationPipeline, CamembertTokenizer
from itertools import chain

# === Initialisation du modèle et du tokenizer CamemBERT ===
model_name = "Jean-Baptiste/camembert-ner-with-dates"
tokenizer = CamembertTokenizer.from_pretrained(model_name)
nlp: TokenClassificationPipeline = pipeline(
    "ner",
    model=model_name,
    tokenizer=tokenizer,
    aggregation_strategy="simple",  # Agrégation des sous-tokens en entités complètes
    device=-1  # -1 pour utiliser le CPU, 0 pour GPU si disponible
)

# === Lecture du fichier texte ===
file_path = "sample_data/APPOLINAIRE_Caligrammes.txt"
with open(file_path, encoding="utf-8") as f:
    texte = f.read()

# === Fonction pour diviser le texte en morceaux (chunks) avec chevauchement ===
def chunk_text(text, chunk_size, overlap):
    start = 0
    while start < len(text):
        end = start + chunk_size
        yield text[start:end]
        start += chunk_size - overlap

# === Conversion d'une entité brute en dictionnaire structuré ===
def get_entity_dict(entity: dict) -> dict:
    return {
        "label": entity['entity_group'],      # Type d'entité (PER, LOC, etc.)
        "text": entity['word'],               # Texte de l'entité
        "jalons": [entity['start'], entity['end']]  # Indices de position dans le texte
    }

# === Traitement du texte avec le modèle NER pour extraire les entités ===
def dico_resultats(texte, nlp: TokenClassificationPipeline, chunk_size=512, overlap=50):
    chunks = list(chunk_text(texte, chunk_size, overlap))  # Diviser le texte
    all_ner_results = list(chain.from_iterable([nlp(chunk) for chunk in chunks]))  # Appliquer le NER sur chaque chunk
    entity_list = [get_entity_dict(entity) for entity in all_ner_results]  # Formatter chaque entité
    return entity_list

# === Exécution de la reconnaissance d'entités nommées ===
liste_entitesnom = dico_resultats(texte, nlp)

# === Affichage des informations utiles ===
tokens = tokenizer.tokenize(texte)  # Tokenisation du texte (subword level)
print(f"Tokens        : {len(tokens)}")  # Nombre total de tokens générés
print(f"Candidats     : {len(liste_entitesnom)}")  # Nombre d'entités détectées
print("-" * 20)
print(f"Ex_tokens     : {tokens[:20]}")  # Affichage des 20 premiers tokens
ex_entites = [{"mot": ent["text"], "type": ent["label"]} for ent in liste_entitesnom[:20]]  # Format simplifié des entités
print(f"Ex_entités    : {ex_entites}")  # Affichage des 20 premières entités extraites

Device set to use cpu


Tokens        : 21295
Candidats     : 732
--------------------
Ex_tokens     : ['▁Rappel', '▁de', '▁votre', '▁demande', ':', '▁Format', '▁de', '▁téléchargement', ':', '▁:', '▁Texte', '▁Vue', 's', '▁1', '▁à', '▁26', '2', '▁sur', '▁26', '2']
Ex_entités    : [{'mot': 'Texte Vues', 'type': 'MISC'}, {'mot': 'Calligrammes', 'type': 'MISC'}, {'mot': 'Guillaume Apollinaire', 'type': 'PER'}, {'mot': 'de La Fresnaye', 'type': 'PER'}, {'mot': 'Apollinaire', 'type': 'PER'}, {'mot': 'Guillaume', 'type': 'PER'}, {'mot': '(1880-1918', 'type': 'DATE'}, {'mot': '(Lausanne)', 'type': 'LOC'}, {'mot': '1952', 'type': 'DATE'}, {'mot': 'La Fresnaye, Roger de', 'type': 'PER'}, {'mot': '(1885-1925', 'type': 'DATE'}, {'mot': 'bnf.fr', 'type': 'LOC'}, {'mot': 'bnf', 'type': 'LOC'}, {'mot': 'atalogue', 'type': 'MISC'}, {'mot': 'bnf', 'type': 'ORG'}, {'mot': 'Français', 'type': 'MISC'}, {'mot': 'in', 'type': 'DATE'}, {'mot': '[Calligrammes', 'type': 'MISC'}, {'mot': 'Collection', 'type': 'MISC'}, {'mot': 'Collect

### Stanza ：

In [1]:
!pip install stanza

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 16.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 49.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 45.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 32.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 62.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 590.6/590.6 kB 25.3 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12

In [2]:
import stanza
def load_stanza_model(lang: str = "fr") -> stanza.Pipeline:
    try:
        nlp = stanza.Pipeline(lang=lang, processors='tokenize,ner')
    except:
        stanza.download(lang=lang, logging_level='DEBUG')
        nlp = stanza.Pipeline(lang=lang, processors='tokenize,ner')
    return nlp

In [3]:
import nltk
import stanza
import re
import json

# Charger le modèle Stanza
def load_stanza_model(lang: str = "fr") -> stanza.Pipeline:
    try:
        nlp = stanza.Pipeline(lang=lang, processors='tokenize,ner')
    except:
        stanza.download(lang=lang, logging_level='DEBUG')
        nlp = stanza.Pipeline(lang=lang, processors='tokenize,ner')
    return nlp

# Lire le texte à partir d'un fichier
def lire_fichier(chemin, is_json=False):
    with open(chemin, encoding='utf-8') as f:
        if is_json:
            return json.load(f)
        else:
            return f.read().strip()

# Obtenir un dictionnaire représentant une entité nommée (modifié selon la structure demandée)
def get_ent_dict(ent) -> dict:
    return {"mot": ent.text, "type": ent.type}

# Traiter le texte et retourner une liste d'entités sans doublons
def dico_resultats(text, lang: str = "fr") -> dict:
    nlp = load_stanza_model(lang=lang)
    doc = nlp(text)

    entites_unique = []
    seen_entities = set()

    for i, ent in enumerate(doc.ents):
        entity_key = (ent.text, ent.type)
        if entity_key not in seen_entities:
            entites_unique.append(get_ent_dict(ent))  # Stocke uniquement {"mot":..., "type":...}
            seen_entities.add(entity_key)

    return entites_unique

# Obtenir les étiquettes au format BIO
def bio_stanza(text: str, lang: str = "fr") -> list[str]:
    nlp = load_stanza_model(lang=lang)
    doc = nlp(text)
    return [
        [token.text, token.ner] for sentence in doc.sentences for token in sentence.tokens
    ]

def remove_punctuation(token):
    return not re.match(r'[\W_]+', token)

if __name__ == "__main__":
    # Chemin du fichier d'entrée
    file_path = "sample_data/APPOLINAIRE_Caligrammes.txt"
    texte = lire_fichier(file_path)

    # Extraire les entités nommées
    entites = dico_resultats(texte, lang="fr")
    print(f"Entités : {len(entites)}")
    print(entites[:20])  # Affiche les 20 premières entités

    # Afficher les tokens obtenus par Stanza
    nlp = load_stanza_model(lang="fr")
    doc = nlp(texte)
    print("\nTokens ：")
    tokens = [token.text for sentence in doc.sentences for token in sentence.tokens]
    print(len(tokens))
    for token in tokens[:20]:
        print(token)



INFO:stanza:Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


INFO:stanza:Downloaded file to /root/stanza_resources/resources.json


INFO:stanza:Loading these models for language: fr (French):
| Processor | Package            |
----------------------------------
| tokenize  | combined           |
| mwt       | combined           |
| ner       | wikinergold_charlm |

INFO:stanza:Using device: cpu
INFO:stanza:Loading: tokenize
INFO:stanza:Loading: mwt
INFO:stanza:Loading: ner
INFO:stanza:Done loading processors!
INFO:stanza:Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


Entités : 584
[{'mot': 'Calligrammes', 'type': 'MISC'}, {'mot': 'Guillaume Apollinaire', 'type': 'PER'}, {'mot': 'La Fresnaye', 'type': 'PER'}, {'mot': 'Apollinaire', 'type': 'PER'}, {'mot': 'Guillaume', 'type': 'PER'}, {'mot': 'Lausanne', 'type': 'LOC'}, {'mot': 'Roger de', 'type': 'PER'}, {'mot': 'Langue : Français', 'type': 'MISC'}, {'mot': 'Format', 'type': 'MISC'}, {'mot': 'Description : Collection : Collection du Bouquet', 'type': 'MISC'}, {'mot': 'Droits : Public domain', 'type': 'MISC'}, {'mot': 'Identifiant', 'type': 'MISC'}, {'mot': 'Bibliothèque nationale de France', 'type': 'LOC'}, {'mot': 'Littérature et art', 'type': 'ORG'}, {'mot': 'Conservation numérique', 'type': 'MISC'}, {'mot': 'OCR', 'type': 'MISC'}, {'mot': 'GUILLAUME APOLLINAIRE', 'type': 'PER'}, {'mot': 'CALLIGRAMMES', 'type': 'PER'}, {'mot': 'M E R M O D', 'type': 'LOC'}, {'mot': 'RENÉ DALIZE', 'type': 'PER'}]


INFO:stanza:Downloaded file to /root/stanza_resources/resources.json
INFO:stanza:Loading these models for language: fr (French):
| Processor | Package            |
----------------------------------
| tokenize  | combined           |
| mwt       | combined           |
| ner       | wikinergold_charlm |

INFO:stanza:Using device: cpu
INFO:stanza:Loading: tokenize
INFO:stanza:Loading: mwt
INFO:stanza:Loading: ner
INFO:stanza:Done loading processors!



Tokens ：
16683
Rappel
de
votre
demande
:
Format
de
téléchargement
:
:
Texte
Vues
1
à
262
sur
262
Nombre
de
pages


### Spacy:

In [ ]:
!python3 -m spacy download fr_core_news_sm
!python3 -m spacy download fr_core_news_lg


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.3/16.3 MB 87.0 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('fr_core_news_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 571.8/571.8 MB 829.5 kB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('fr_core_news_lg')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [ ]:
import json
import spacy

# Charger le modèle SpaCy français
nlp = spacy.load('fr_core_news_sm')

# Chemin du fichier à analyser
file_path = "sample_data/APPOLINAIRE_Caligrammes.txt"

# Lire le contenu du fichier
with open(file_path, "r", encoding="utf-8") as f:
    contenu = f.read()

# Analyse avec SpaCy
doc = nlp(contenu)

# Compter les tokens (hors espaces)
nb_tokens = len([token for token in doc if not token.is_space])

# Compter les entités nommées
nb_entites = len(doc.ents)

# Structurer les entités nommées
entites = [{"texte": ent.text, "type": ent.label_} for ent in doc.ents]

# Résultat structuré
resultat = {
    "fichier": file_path,
    "nombre de tokens": nb_tokens,
    "nombre d'entités nommées": nb_entites,
    "entités nommées": entites
}

# Sauvegarder en JSON
output_path = "resultat_spacy_APPOLINAIRE.json"
with open(output_path, "w", encoding="utf-8") as json_file:
    json.dump(resultat, json_file, ensure_ascii=False, indent=4)

# Affichage résumé
print(f"Fichier analysé : {file_path}")
print(f"Nombre de tokens : {nb_tokens}")
print(f"Nombre d'entités nommées : {nb_entites}")
print(f"Résultat enregistré dans : {output_path}")


Fichier analysé : sample_data/APPOLINAIRE_Caligrammes.txt
Nombre de tokens : 16834
Nombre d'entités nommées : 1260
Résultat enregistré dans : resultat_spacy_APPOLINAIRE.json


In [ ]:
import json
import spacy

# Charger le modèle français large de spaCy
nlp = spacy.load('fr_core_news_lg')

# Fichier à traiter
file_path = "sample_data/APPOLINAIRE_Caligrammes.txt"

# Lire le contenu du fichier
with open(file_path, "r", encoding="utf-8") as f:
    contenu = f.read()

# Traitement avec spaCy
doc = nlp(contenu)

# Nombre de tokens (hors espaces)
nb_tokens = len([token for token in doc if not token.is_space])

# Nombre d'entités nommées
nb_entites = len(doc.ents)

# Structuration des entités
entites = [{"texte": ent.text, "type": ent.label_} for ent in doc.ents]

# Structuration du résultat
resultat = {
    "fichier": file_path,
    "nombre de tokens": nb_tokens,
    "nombre d'entités nommées": nb_entites,
    "entités nommées": entites
}

# Sauvegarde JSON
output_path = "resultat_lg_APPOLINAIRE.json"
with open(output_path, "w", encoding="utf-8") as json_file:
    json.dump(resultat, json_file, ensure_ascii=False, indent=4)

# Affichage résumé
print(f"Fichier analysé : {file_path}")
print(f"Nombre de tokens : {nb_tokens}")
print(f"Nombre d'entités nommées : {nb_entites}")
print(f"Résultat enregistré dans : {output_path}")



Fichier analysé : sample_data/APPOLINAIRE_Caligrammes.txt
Nombre de tokens : 16834
Nombre d'entités nommées : 1171
Résultat enregistré dans : resultat_lg_APPOLINAIRE.json


### Flaubert：